# Project: DaXiGua Agency

In [11]:
from State import State
from Movement_evaluation import evaluate_by_gravity
import numpy as np
from Ball import Ball
import time

In [232]:
from State import State
from Ball import Ball, Position, Velocity
import copy
import numpy as np


def check_converge(frames, tolerance=5):
    """
    Check if the final state has converged
    
    frames: List[State], the frames stocked in order of time
    """
    
    last_frames = frames[-10:]
    current_frame = last_frames[-1]
    for frame in last_frames:
        if len(frame.balls) != len(current_frame.balls):
            return False
        
        for b1, b2 in zip(frame.balls, current_frame.balls):
#             print(np.linalg.norm(b1.position - b2.position), b1.position, b2.position)
            if np.linalg.norm(b1.position - b2.position) > tolerance:
                return False
    print("CONVERGE")
    return True
    

def evaluate_by_gravity(state, plot=False, dt=0.1, check_converge_step = 10, protection_time_limit = 30):
    """
    state: State, the initial state
    plot: bool, if plot the progress of the movement
    return: State, the final converged state
    
    implement the movement of the balls in the state by the effect of gravity
    """
    
    g = -39.8
    amortize_factor = 1.5  # further tuning needed
    collision_factor = 0.5  # further tuning needed

    screen_limit = np.array([state.screen_x, state.screen_y])

    t = 0

    frames = [state]  # store the frames of evaluation
    converged = False  

    balls = state.balls
    
    count = 0
    
    while not converged:

        N = len(balls)

        for i in range(N):
            b = balls[i]
            f = np.array([0, 1*g])

    #         Update the velocity
            b.velocity = (1 - amortize_factor * dt) * b.velocity + dt * f

    #         Update the position
            b.position = b.position + b.velocity * dt

    #         Check collision with borders
            for j in range(2):
                if b.position[j] < b.radius:
                    b.velocity[j] = 0
                    b.position[j] = b.radius          
                    
                if j == 0 and b.position[j] > screen_limit[j] - b.radius:
                    b.velocity[j] = 0
                    b.position[j] = screen_limit[j] - b.radius

    #         Check collisions between balls
        i = 0
        while i < len(balls):
            j = i + 1
            while j < len(balls):
                ball_1 = balls[i]
                ball_2 = balls[j]
                
                if ( np.linalg.norm(ball_1.position - ball_2.position) <= ball_1.radius + ball_2.radius ):
                    
                    m1 = ball_1.radius
                    m2 = ball_2.radius
                    if m1 != m2:
                        mid_point = (1/(m1 + m2)) * (m2 * ball_1.position +  m1 * ball_2.position)
                        u = (ball_2.position - ball_1.position)  # uniform vector from ball 1 to ball 2
                        u = u / np.linalg.norm(u)
                        #  Update the positions after collision
                        ball_1.position = mid_point - ball_1.radius * u
                        ball_2.position = mid_point + ball_2.radius * u

                        # Update the velocity of balls after collsion
                        # divide the velocity to two dimension : u and n

                        v1_u = np.dot(ball_1.velocity, u) * u
                        v1_n = ball_1.velocity - v1_u

                        v2_u = np.dot(ball_2.velocity, u) * u
                        v2_n = ball_2.velocity - v2_u

                        # the velocity of direction n does not change, but in direction u they exchange


                        v1_u_after = collision_factor * ((m1 - m2) * v1_u + 2 * m2 * v2_u)/(m1 + m2)
                        v2_u_after = collision_factor * ((m2 - m1) * v2_u + 2 * m1 * v1_u)/(m1 + m2)

                        ball_1.velocity = amortize_factor*v1_n + v2_u_after
                        ball_2.velocity = amortize_factor*v2_n + v1_u_after
                        j += 1
                  
                    else:
                        #  Form a new ball with larger radius
                        mid_point = (1/(m1 + m2)) * (m2 * ball_1.position +  m1 * ball_2.position)
                        
                        del balls[j]
                        balls[i].position = mid_point
                        balls[i].velocity = np.array([0, 0])
                        balls[i].radius = 1.5*balls[i].radius  # TO MODIFIED WITH GAME SETTING
                else:
                    j += 1
            i += 1
     
        for i in range(len(balls)):
            b = balls[i]
            for j in range(2):
                if b.position[j] < b.radius:
                    b.velocity[j] = 0
                    b.position[j] = b.radius          

                if j == 0 and b.position[j] > screen_limit[j] - b.radius:
                    b.velocity[j] = 0
                    b.position[j] = screen_limit[j] - b.radius
            
        count += 1
        for i in range(len(balls)):
            b = balls[i]
            if np.linalg.norm(b.velocity) < 0.5 * -g * dt:
                b.velocity = np.array([0,0])

        if plot:
            if count % 5 == 0:
                state.plot_state()
        
        frames.append( copy.deepcopy(state) )
        if len(frames) >= check_converge_step and len(frames) % check_converge_step == 0:
            converged = check_converge(frames)
        t += dt
        if t > protection_time_limit:  # protection, need more tuning
            break;
    return state

## Test the movement evaluation

运动规则如下：

1. 所有球受重力影响下坠
2. 与墙壁和地板的碰撞无反弹
3. 有空气阻力，小球下落速度越大阻力越大
4. 所有球之间的碰撞有一定的速度损失，并非完全弹性碰撞
5. 两个同样大小的小球相撞后合成新的小球，新的小球半径增大50% （这一条需根据具体游戏规则再进行修改）

In [239]:
state_test_1 = State(300, 
                     600, 
                     [Ball(np.array([100, 355]), np.array([-10, -10.0]), 20, color=(125, 0, 0)), 
                      Ball(np.array([50, 20]), np.array([20, 0]), 30, color=(126, 120, 0)),
                      Ball(np.array([60, 215]), np.array([-10, -10.0]), 30, color=(125, 120, 0)), 
                      Ball(np.array([100, 20]), np.array([0, 0]), 20, color=(126, 20, 0)), 
                      Ball(np.array([250, 156]), np.array([-10, 0]), 40, color=(0, 20, 126)),
                      Ball(np.array([90, 56]), np.array([0, 0]), 50, color=(10, 90, 55)), 
                      Ball(np.array([240, 72]), np.array([0, 10]), 40, color=(0, 20, 126)),
                      Ball(np.array([150, 156]), np.array([-10, 3]), 10, color=(0, 90, 26)), 
                      Ball(np.array([140, 272]), np.array([29, 10]), 10, color=(0, 90, 26)),], 
                     500)

### Initial State
<img src = "figures/sample_1.PNG" width="200"/>

In [240]:
state_test_1.plot_state()

Position, x: 100, y: 355 
Velocity, vx: -10.0, vy: -10.0 
Radius: 20

Position, x: 50, y: 20 
Velocity, vx: 20, vy: 0 
Radius: 30

Position, x: 60, y: 215 
Velocity, vx: -10.0, vy: -10.0 
Radius: 30

Position, x: 100, y: 20 
Velocity, vx: 0, vy: 0 
Radius: 20

Position, x: 250, y: 156 
Velocity, vx: -10, vy: 0 
Radius: 40

Position, x: 90, y: 56 
Velocity, vx: 0, vy: 0 
Radius: 50

Position, x: 240, y: 72 
Velocity, vx: 0, vy: 10 
Radius: 40

Position, x: 150, y: 156 
Velocity, vx: -10, vy: 3 
Radius: 10

Position, x: 140, y: 272 
Velocity, vx: 29, vy: 10 
Radius: 10

----------------------------------------------------------------


In [244]:
start_time = time.time()
evaluate_by_gravity(state_test_1, plot=False)
end_time  = time.time()
print("Using ", end_time - start_time, "s")

CONVERGE
Using  0.008975505828857422 s


In [245]:
state_test_1.plot_state()

Position, x: 117.79010650587355, y: 30.0 
Velocity, vx: 0, vy: 0 
Radius: 30.0

Position, x: 45.0, y: 45.08247871370489 
Velocity, vx: 0, vy: 0 
Radius: 45.0

Position, x: 201.99018048560558, y: 60.0 
Velocity, vx: 0, vy: 0 
Radius: 60.0

Position, x: 250.0, y: 157.86137296944818 
Velocity, vx: 0.0, vy: -5.930198589860605 
Radius: 50

Position, x: 285.0, y: 15.0 
Velocity, vx: 0, vy: 0 
Radius: 15.0

----------------------------------------------------------------


### State after evaluation
<img src = "figures/sample_1_res.PNG" width="200"/>

In [250]:
state_test_1.balls.append( Ball(np.array([150, 550]), np.array([0, 0.0]), 33, color=(150, 100, 247)) )

In [251]:
state_test_1.plot_state()

Position, x: 117.79303504751348, y: 30.0 
Velocity, vx: 0, vy: 0 
Radius: 30.0

Position, x: 45.0, y: 45.0 
Velocity, vx: 0, vy: 0 
Radius: 45.0

Position, x: 201.99393962866273, y: 60.0 
Velocity, vx: 0, vy: 0 
Radius: 60.0

Position, x: 250.0, y: 157.8637268704916 
Velocity, vx: 0.0, vy: -5.929169550405437 
Radius: 50

Position, x: 285.0, y: 15.0 
Velocity, vx: 0, vy: 0 
Radius: 15.0

Position, x: 33.0, y: 121.3024995157705 
Velocity, vx: 0.0, vy: -2.878243473715032 
Radius: 33

Position, x: 150, y: 550 
Velocity, vx: 0.0, vy: 0.0 
Radius: 33

----------------------------------------------------------------


In [252]:
evaluate_by_gravity(state_test_1, plot=False)

CONVERGE


In [253]:
state_test_1.plot_state()

Position, x: 118.57646313726163, y: 30.0 
Velocity, vx: 0, vy: 0 
Radius: 30.0

Position, x: 45.0, y: 45.0 
Velocity, vx: 0, vy: 0 
Radius: 45.0

Position, x: 208.95328574436377, y: 60.0 
Velocity, vx: 0, vy: 0 
Radius: 60.0

Position, x: 250.0, y: 160.68734309943397 
Velocity, vx: 0.0, vy: -4.985689289292827 
Radius: 50

Position, x: 285.0, y: 15.0 
Velocity, vx: 0, vy: 0 
Radius: 15.0

Position, x: 112.2178100593409, y: 110.02564092219997 
Velocity, vx: -2.5632457433005644, vy: -5.77583870485226 
Radius: 49.5

----------------------------------------------------------------


In [10]:
class Game(object):
    
    """
    Implement the environment of the Game
    """
    
    def __init__(self, screen_x, screen_y, ball_setting ):
        """
        screen_x: float, the width of the screen
        screen_y: float, the height of the screen
        ball_setting: Dict( radius: float, reward: float ), the sizes of balls and corresponding rewards used in the function
        """
        
        self.screen_x = screen_x
        self.screen_y = screen_y
        self.ball_setting = ball_setting
        self.current_state = None  # TO DO
        self.init_state()
        
#         --TO ADD MORE--
        
    def init_state(self):
#         --TO DO--
        return 

    def check_fin(self):
#         --TO DO--
        return 
    
    def calculate_reward():
#         --TO DO--
        return 

#     ---TO ADD MORE---